Q1. **Data Preprocessing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install beautifulsoup4 nltk

In [ ]:
import os
import nltk
import random
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:

# Function to perform data preprocessing
def preprocess_text(input_text):

    # a. Convert text to lowercase
    lowercase_text = input_text.lower()

    # b. Tokenization using NLTK
    tokens = word_tokenize(lowercase_text)

    # c. Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # d. Remove punctuation
    no_punctuations = [word for word in filtered_tokens if word.isalnum()]

    # e. Remove blank space tokens
    no_blank_space = [word for word in no_punctuations if word.strip() != '']

    # Combine the tokens into a preprocessed text
    preprocessed_text = ' '.join(no_blank_space)

    return preprocessed_text


def preprocessed_folder(folder_path, output_directory):

    for count, filename in enumerate(os.listdir(folder_path), start=1):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            output_path = os.path.join(output_directory, f'processed_{filename}')

            with open(file_path, 'r') as file, open(output_path, 'w') as output_file:
                text = file.read()
                preprocessed_text = preprocess_text(text)
                output_file.write(preprocessed_text)

            # Print contents for the first 5 files
            if count <= 5:
                print(f"Original Content of {filename} (File {count}):\n{text}\n")
                print(f"Preprocessed Content of {filename} (File {count}):\n{preprocessed_text}\n")


# Folder containing text files
folder_path = '/content/drive/MyDrive/IR_Assignments/text_files_IR'

# Preprocessed files are saved here
output_directory = '/content/drive/MyDrive/IR_Assignments/output_directory_IR_1'

# Preprocessed folder and prints its contents
preprocessed_folder(folder_path, output_directory)


Q2. **Unigram Inverted Index**

In [ ]:
import pickle

# Function to create a unigram inverted index
def create_inverted_index(dataset_path):
    inverted_index = {}

    # Iterate through each preprocessed file in the dataset
    for filename in os.listdir(dataset_path):
        if filename.startswith('processed_') and filename.endswith('.txt'):
            file_path = os.path.join(dataset_path, filename)

            with open(file_path, 'r') as file:
                text = file.read()
                tokens = word_tokenize(text)

                # Removing duplicate tokens
                unique_tokens = list(set(tokens))

                # Updating the inverted index for each term in the document
                for i in unique_tokens:

                  if i not in inverted_index:
                      inverted_index[i] = [filename]
                  else:
                      inverted_index[i].append(filename)

    return inverted_index

# Specify the path to your preprocessed files directory
preprocessed_files_directory = '/content/drive/MyDrive/IR_Assignments/output_directory_IR_1'

# Create the unigram inverted index
unigram_inverted_index = create_inverted_index(preprocessed_files_directory)

# Printing the unigram inverted index
# for term, documents in unigram_inverted_index.items():
#    print(f"{term}: {documents}")

# print('\n')

# Save the unigram inverted index using pickle
output_index_path = '/content/drive/MyDrive/IR_Assignments/unigram_inverted_index/unigram_inverted_index.pkl'
with open(output_index_path, 'wb') as output_file:
    pickle.dump(unigram_inverted_index, output_file)

print(f"Unigram inverted index saved to {output_index_path}")

# Load the unigram inverted index using pickle
input_index_path = '/content/drive/MyDrive/IR_Assignments/unigram_inverted_index/unigram_inverted_index.pkl'
with open(input_index_path, 'rb') as input_file:
    loaded_unigram_inverted_index = pickle.load(input_file)

# Now, loaded_unigram_inverted_index contains the previously saved index and then printing it
print(f"Unigram inverted index loaded from {input_index_path}")
print(loaded_unigram_inverted_index)

**Boolean Queries**

In [23]:
import pickle
import string

# stopwords - nltk
stop_words = set(stopwords.words('english'))

def load_inverted_index(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token.isalnum() and token not in stop_words]

def parse_operations(operations_str):
    return operations_str.split(', ')

def execute_query(terms, operations, inverted_index):
    result_docs = set(inverted_index.get(terms[0], []))
    for i, term in enumerate(terms[1:], start=1):
        current_docs = set(inverted_index.get(term, []))
        operation = operations[i-1] if i-1 < len(operations) else None

        if operation == 'AND':
            result_docs &= current_docs
        elif operation == 'OR':
            result_docs |= current_docs
        elif operation == 'AND NOT':
            result_docs -= current_docs
        elif operation == 'OR NOT':
          #dont understand how it works
            continue

    return sorted(result_docs)

def format_output(query_number, terms, operations, results):
    query_str = ' '.join([f"{terms[i]} {operations[i]}" if i < len(operations) else terms[i] for i in range(len(terms))])
    result_str = ', '.join(results) if results else "No documents found"
    print(f"Query {query_number}: {query_str}")
    print(f"Number of documents retrieved: {len(results)}")
    print(f"Documents: {result_str}")

def main():
    inverted_index = load_inverted_index('/content/drive/MyDrive/IR_Assignments/unigram_inverted_index/unigram_inverted_index.pkl')
    num_queries = int(input("Enter the number of queries: "))
    queries = []
    operations_list = []

    # Collect queries and operations
    for i in range(num_queries):
        query = input(f"Enter query {i+1}: ")
        operations_input = input("Enter operations: ")
        queries.append((query, operations_input))

    # Process each query and store results
    results = []
    for i, (query, operations_input) in enumerate(queries, start=1):
        terms = preprocess(query)
        operations = parse_operations(operations_input)

        if len(operations) != len(terms) - 1:
            results.append(f"Error: Number of operations should be one less than the number of terms for Query {i}")
            continue

        result_docs = execute_query(terms, operations, inverted_index)
        results.append(format_output(i, terms, operations, result_docs))

    # Print all results together
    for result in results:
      if(result):
        print(result)

main()

Enter the number of queries: 2
Enter query 1: Car in a Canister
Enter operations: OR
Enter query 2: Fits can CAR in a Canister
Enter operations: OR, OR
Query 1: car OR canister
Number of documents retrieved: 6
Documents: processed_file166.txt, processed_file174.txt, processed_file264.txt, processed_file542.txt, processed_file746.txt, processed_file886.txt
Query 2: fits OR car OR canister
Number of documents retrieved: 66
Documents: processed_file107.txt, processed_file110.txt, processed_file118.txt, processed_file166.txt, processed_file169.txt, processed_file174.txt, processed_file176.txt, processed_file179.txt, processed_file187.txt, processed_file189.txt, processed_file221.txt, processed_file242.txt, processed_file264.txt, processed_file285.txt, processed_file302.txt, processed_file324.txt, processed_file341.txt, processed_file346.txt, processed_file364.txt, processed_file38.txt, processed_file39.txt, processed_file431.txt, processed_file454.txt, processed_file460.txt, processed_file

Q3. **Positional Index**

In [ ]:
import os
import pickle

def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token.isalnum() and token not in stop_words]

def create_positional_index(docs_folder):
    positional_index = {}

    # Loop through each document in the dataset
    for filename in os.listdir(docs_folder):
        file_path = os.path.join(docs_folder, filename)

        # Read the content of the document
        with open(file_path, 'r') as file:
            text = file.read()

        # Preprocess the text and get the terms
        terms = preprocess(text)

        # Update the positional index with terms from this document
        for position, term in enumerate(terms):
            if term not in positional_index:
                positional_index[term] = {}
            if filename not in positional_index[term]:
                positional_index[term][filename] = []
            positional_index[term][filename].append(position)

    return (positional_index)

# folder containing the preprocessed files
preprocessed_files_folder = '/content/drive/MyDrive/IR_Assignments/output_directory_IR_1'
positional_index = create_positional_index(preprocessed_files_folder)

def save_positional_index(positional_index, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(positional_index, file)

# Save the positional index to a file
save_positional_index(positional_index, '/content/drive/MyDrive/IR_Assignments/positional_index/positional_index.pkl')

def load_positional_index(file_path):
    with open(file_path, 'rb') as file:
        positional_index = pickle.load(file)
    return positional_index

# Load the positional index from a file
positional_index = load_positional_index('/content/drive/MyDrive/IR_Assignments/positional_index/positional_index.pkl')
print(positional_index)



**Phrase Queries**

In [19]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token.isalnum() and token not in stop_words]

# Function to load the positional index
def load_positional_index(file_path):
    with open(file_path, 'rb') as file:
        positional_index = pickle.load(file)
    return positional_index


def execute_phrase_query(query, positional_index):
    # Preprocess the query
    terms = preprocess(query)

    # Check if the first term is in the positional index
    if terms[0] not in positional_index:
        return set()

    # Initialize potential result documents with the first term's documents
    potential_docs = positional_index[terms[0]].keys()

    # This will hold the final document IDs where the phrase is found
    final_docs = set()

    for doc in potential_docs:
        # Lists to store term positions for each term in the query
        term_positions = [positional_index[term][doc] for term in terms if term in positional_index and doc in positional_index[term]]

        # Skip documents that don't have all terms
        if len(term_positions) != len(terms):
            continue

        # Check for consecutive positions
        # Iterate over each position of the first term
        for start_pos in term_positions[0]:
            # Check if for each subsequent term there's a position = previous position + 1
            if all((start_pos + i in positions) for i, positions in enumerate(term_positions)):
                final_docs.add(doc)
                break  # No need to check other start positions if one sequence matches

    return final_docs


# Main function
def main():
    # Load the positional index
    positional_index_path = '/content/drive/MyDrive/IR_Assignments/positional_index/positional_index.pkl'
    positional_index = load_positional_index(positional_index_path)

    num_queries = int(input("Enter the number of queries: "))

    # Lists to store queries and results
    queries = []
    all_results = []

    # Collect all queries first
    for i in range(num_queries):
        query_input = input(f"Enter phrase query {i+1}: ")
        queries.append(query_input)

    # Process each query
    for query in queries:
        results = execute_phrase_query(query, positional_index)
        all_results.append((query, results))

    # Display all results together
    for i, (query, results) in enumerate(all_results):
    # Print the query number and the number of documents retrieved
      print(f"Number of documents retrieved for query {i+1} using positional index: {len(results)}")

      # Check if there are results
      if results:
          # If there are results, join them into a string and print
          documents_str = ', '.join(results)
          print(f"Names of documents retrieved for query {i+1} using positional index: {documents_str}")
      else:
          print(f"Names of documents retrieved for query {i+1} using positional index: No documents found")

main()


Enter the number of queries: 1
Enter phrase query 1: it is good
Number of documents retrieved for query 1 using positional index: 204
Names of documents retrieved for query 1 using positional index: processed_file2.txt, processed_file539.txt, processed_file541.txt, processed_file748.txt, processed_file731.txt, processed_file160.txt, processed_file655.txt, processed_file984.txt, processed_file532.txt, processed_file715.txt, processed_file103.txt, processed_file713.txt, processed_file28.txt, processed_file795.txt, processed_file118.txt, processed_file9.txt, processed_file137.txt, processed_file586.txt, processed_file777.txt, processed_file610.txt, processed_file974.txt, processed_file8.txt, processed_file497.txt, processed_file918.txt, processed_file240.txt, processed_file325.txt, processed_file885.txt, processed_file896.txt, processed_file282.txt, processed_file794.txt, processed_file111.txt, processed_file44.txt, processed_file772.txt, processed_file919.txt, processed_file844.txt, proc